In [0]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

In [0]:
pip install markovify

In [0]:
pip install nltk

In [0]:
import markovify
import nltk
import re

In [0]:
nltk.download('punkt')

In [0]:
from urllib import request
from bs4 import BeautifulSoup
from urllib import request

In [0]:
## switch to directory with chapterize.py and .txt files ##
cd drive/My Drive/PoeMelBot

In [0]:
## Cleaning ##
## get edgar allan poe text and clean ##
url = "http://www.gutenberg.org/cache/epub/10031/pg10031.txt"
poe_txt_raw = BeautifulSoup(request.urlopen(url)).get_text()
poe_txt_raw = poe_txt_raw.replace("*       *       *       *       *" , "\n")
with open("poe.txt" , 'w') as f:
  f.write(poe_txt_raw)

!python chapterize.py --nochapters poe.txt  ## remove chapter number, Gutenberg license text, etc ##

with open("poe-extracted.txt") as f:
  poe_txt_fil = f.read()
raven = poe_txt_raw[poe_txt_raw.find("THE RAVEN"):poe_txt_raw.find("Published, 1845.")]
poe_txt_fil = raven + poe_txt_fil

with open("poe-extracted.txt" , 'w') as f:
  f.write(poe_txt_fil)

## remove unnecessary Notes from the text ##

start = poe_txt_fil.find("NOTES")
end = poe_txt_fil.find("LENORE.")
temp = poe_txt_fil[start:end]
poe_txt_fil = poe_txt_fil.replace(temp , "\n")

start = poe_txt_fil.find("NOTES")
end = poe_txt_fil.find("AL AARAAF")
temp = poe_txt_fil[start:end]
poe_txt_fil = poe_txt_fil.replace(temp , "\n")

start = poe_txt_fil.find("NOTES")
end = poe_txt_fil.find("THE VILLAGE STREET")
temp = poe_txt_fil[start:end]
poe_txt_fil = poe_txt_fil.replace(temp , "\n")

start = poe_txt_fil.find("ISRAFEL")
end = poe_txt_fil.find("SONG")
temp = poe_txt_fil[start:end]
poe_txt_fil = poe_txt_fil.replace(temp , "\n")

start = poe_txt_fil.find("NOTES")
temp = poe_txt_fil[start:]
poe_txt_fil = poe_txt_fil.replace(temp , "\n")

with open("poe-filtered.txt" , 'w') as f:
  f.write(poe_txt_fil)
poe_txt_caps = re.sub(r'\b[A-Z]+\b', '', poe_txt_fil)
with open("poe-filtered.txt" , 'w') as f:
  f.write(poe_txt_caps)

## more cleaning done in poe-mel-filter.R part 1 ##

In [0]:
## Cleaning ##
## get herman melville text and clean ##
url = "http://www.gutenberg.org/cache/epub/12841/pg12841.txt"
mel_txt_raw = BeautifulSoup(request.urlopen(url)).get_text()
with open("mel.txt" , 'w') as f:
  f.write(mel_txt_raw)

!python chapterize.py --nochapters mel.txt  ## remove chapter number, Gutenberg license text, etc ##

with open("mel-extracted.txt") as f:
  mel_txt2 = f.read()

## remove unnecessary notes and titles ## 
  
start = mel_txt2.find("Introductory Note")
end = mel_txt2.find("Since as in night's deck")
mel_txt2 = mel_txt2.replace(mel_txt2[start:end] , "\n")

mel_txt2 = mel_txt2.replace("Sea Pieces" , ".")
mel_txt2 = mel_txt2.replace("Poems From Timoleon" , ".")
mel_txt2 = mel_txt2.replace("Poems From Battle Pieces" , ".")
mel_txt2 = mel_txt2.replace("Poems From Mardi" , ".")
mel_txt2 = mel_txt2.replace("Poems From Clarel" , ".")

start = mel_txt2.find("SUPPLEMENT")
end = mel_txt2.find("THE PORTENT")
mel_txt2 = mel_txt2.replace(mel_txt2[start:end] , "\n")

mel_txt2 = re.sub(r'\b[A-Z]+\b', '', mel_txt2)

with open("mel-filtered.txt" , 'w') as f:
  f.write(mel_txt2)

## more cleaning in poe-mel-filter.R part 2 ##

In [0]:
## Herman Melville Markov model ##

with open("mel-filtered_2.txt") as f:
  mel_fil = f.read()
mel_model = markovify.Text(mel_fil , state_size = 2)
mel_json = mel_model.to_json()
with open("mel_model.json" , 'w') as f:
  f.write(mel_json)

## example instance of the markov chain ##
print(mel_model.make_sentence(tries = 150 , max_overlap_ratio = 0.6 , max_chars = 140).replace("." , "\n"))

In [0]:
## Edgar Allan Poe Markov model ##

with open("poe-filtered_2.txt") as f:
  poe_txt_2 = f.read()
poe_model = markovify.Text(poe_txt_2 , state_size = 2)
poe_json = poe_model.to_json()
with open("poe_model.json" , 'w') as f:
  f.write(poe_json)

## example instance of the markov chain ##
print(poe_model.make_sentence(max_chars = 200 , max_overlap_ratio = 0.25 , tries = 150).replace("." , "\n"))

In [0]:
## check if both the corpora have comparable number of words to make a combined model ##

from nltk import word_tokenize
poe_tokens = word_tokenize(poe_txt_2)
mel_tokens = word_tokenize(mel_fil)
print([len(poe_tokens) , len(mel_tokens)])
intersect = (set(poe_tokens) & set(mel_tokens))
print(len(intersect))
unique_poe = len(poe_tokens) - len(intersect)
unique_mel = len(mel_tokens) - len(intersect)
print([unique_poe , unique_mel])

## seems good enough ##

In [0]:
## combined markov model ##

poe_mel_model = markovify.combine([poe_model , mel_model])
poe_mel_json = poe_mel_model.to_json()
with open("poe_mel_model.json" , 'w') as f:
  f.write(poe_mel_json)

## example instance of the markov chain ##
print(poe_mel_model.make_sentence(tries = 150 , max_overlap_ratio = 0.5 , max_chars = 140).replace("." , "\n"))

In [0]:
## single markov model, randomized lines ##
## randomization of text done in poe-mel-filter.R part 3 ##

with open("poe-mel-rand.txt") as f:
  poe_mel_txt = f.read()
poe_mel_rand_model = markovify.Text(poe_mel_txt , state_size = 2)
poe_mel_rand_json = poe_mel_rand_model.to_json()
with open("poe_mel_rand_model.json" , 'w') as f:
  f.write(poe_mel_rand_json)

## example instance of the markov chain ##
print(poe_mel_rand_model.make_sentence(tries = 150 , max_overlap_ratio = 0.75 , max_chars = 140).replace("." , "\n"))